## Debugging With Git Bisect

You can use

``` bash
git bisect
```

to find out which commit caused a bug.

### An example repository

In a nice open source example, I found an arbitrary exemplar on github

In [231]:
import os
top_dir = os.getcwd()
git_dir = os.path.join(top_dir, 'learning_git')
working_dir=os.path.join(git_dir, 'git_example')
os.chdir(working_dir)os.chdir(git_dir)

In [232]:
%%bash
rm -rf bisectdemo
git clone git@github.com:shawnsi/bisectdemo.git

Cloning into 'bisectdemo'...


In [233]:
bisect_dir=os.path.join(git_dir,'bisectdemo')
os.chdir(bisect_dir)

In [234]:
%%bash
python squares.py 2 # 4

4


This has been set up to break itself at a random commit, and leave you to use
bisect to work out where it has broken:

In [235]:
%%bash 
./breakme.sh > break_output

error: branch 'buggy' not found.
Switched to a new branch 'buggy'


Which will make a bunch of commits, of which one is broken, and leave you in the broken final state

In [236]:
python squares.py 2 # Error message

SyntaxError: invalid syntax (<ipython-input-236-69f578907137>, line 1)

In [241]:
### Bisecting manually

In [237]:
%%bash
git bisect start
git bisect bad # We know the current state is broken
git checkout master
git bisect good # We know the master branch state is OK

Your branch is up-to-date with 'origin/master'.
Bisecting: 500 revisions left to test after this (roughly 9 steps)
[97b8922da658f4adbe0e8d1262062d171e2f1683] Comment 500


Switched to branch 'master'


Bisect needs one known good and one known bad commit to get started

### Solving Manually

``` bash
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # Crash
git bisect bad
python squares.py 2 # Crash
git bisect bad
python squares.py 2 # Crash
git bisect bad
python squares.py 2 #Crash
git bisect bad
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
```


And eventually:

``` bash
git bisect good
    Bisecting: 0 revisions left to test after this (roughly 0 steps)

python squares.py 2
    4

git bisect good
2777975a2334c2396ccb9faf98ab149824ec465b is the first bad commit
commit 2777975a2334c2396ccb9faf98ab149824ec465b
Author: Shawn Siefkas <shawn.siefkas@meredith.com>
Date:   Thu Nov 14 09:23:55 2013 -0600

    Breaking argument type

```

``` bash
git bisect end
```

### Solving automatically

If we have an appropriate unit test, we can do all this automatically:

In [242]:
%%bash
git bisect start
git bisect bad HEAD # We know the current state is broken
git bisect good master # We know master is good
git bisect run python squares.py 2

Bisecting: 500 revisions left to test after this (roughly 9 steps)
[97b8922da658f4adbe0e8d1262062d171e2f1683] Comment 500
running python squares.py 2
4
Bisecting: 250 revisions left to test after this (roughly 8 steps)
[f10ce756d4cce15135566ecfc2c5d85004ab191f] Comment 750
running python squares.py 2
4
Bisecting: 125 revisions left to test after this (roughly 7 steps)
[ec9e48e76383217dc2dd5dd8bb11ac3825e473f5] Comment 875
running python squares.py 2
4
Bisecting: 62 revisions left to test after this (roughly 6 steps)
[3c1afca389c1cbb2cf7bfea3afc04accf17265e5] Comment 937
running python squares.py 2
Bisecting: 31 revisions left to test after this (roughly 5 steps)
[13dae55a5d37c6bbe477f555e7c829607e1dbbe9] Comment 905
running python squares.py 2
Bisecting: 15 revisions left to test after this (roughly 4 steps)
[76dd89679d416901181e1c70dcf8b674f3c72f2e] Comment 889
running python squares.py 2
Bisecting: 7 revisions left to test after this (roughly 3 steps)
[2777975a2334c2396ccb9faf98ab149

Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'
Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'
Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'
Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'


Boom!